In [19]:
import requests
from IPython.core.display import display, HTML
from bs4 import BeautifulSoup
from personal import *

url = "http://127.0.0.1:5000"
# url = "https://turnitin-api.herokuapp.com"


s = requests.Session()

In [20]:
login_result = s.post(url + "/login", json={
    "email": USERNAME,
    "password": PASSWORD
})
auth = login_result.json()

classes_result = s.post(url + "/classes", json=auth)
classes = classes_result.json()
for c in classes:
    if "World Lit" in c["title"]:
        world_lit = c
print(world_lit)

first_class_data = dict(auth, url=world_lit["url"])
assignments_result = s.post(url + "/assignments", json=first_class_data)
assignments = assignments_result.json()
assignments[-6]

{'title': 'World Literature: Period 5', 'url': 'https://www.turnitin.com/s_class_portfolio.asp?r=29.1421237500028&svr=23&lang=en_us&aid=5245&cid=22703844'}


{'aid': '93021867',
 'dates': {'due': '06/30/2020 23:59:00',
  'post': '06/12/2020 00:00:00',
  'start': '06/04/2020 15:33:00'},
 'file': 'void',
 'info': 'No additional instructions have been entered for this assignment. Late paper submissions are allowed for this assignment. Resubmissions are not allowed for this assignment. Similarity reports are available immediately after submission.',
 'oid': 'void',
 'submission': 't_submit.asp?r=3.18340456629862&svr=41&lang=en_us&aid=93021867',
 'title': "Kento's Test Assignment (for Kento only)"}

In [16]:
len(assignments)

29

In [21]:
assignments[-6]

{'aid': '93021867',
 'dates': {'due': '06/30/2020 23:59:00',
  'post': '06/12/2020 00:00:00',
  'start': '06/04/2020 15:33:00'},
 'file': 'void',
 'info': 'No additional instructions have been entered for this assignment. Late paper submissions are allowed for this assignment. Resubmissions are not allowed for this assignment. Similarity reports are available immediately after submission.',
 'oid': 'void',
 'submission': 't_submit.asp?r=3.18340456629862&svr=41&lang=en_us&aid=93021867',
 'title': "Kento's Test Assignment (for Kento only)"}

In [44]:
ass = assignments[-6]
aid = ass["aid"]
cookies = auth["auth"]
author_first = "Kento"
author_last = "Nishi"
with open("../SampleSubmission.docx", 'rb') as fin:
    filebytes = fin.read()
files = {"SampleSubmission.docx": open("../SampleSubmission.docx", 'rb')}
submission_title = "Custom turnitin API"

In [45]:
referer_url = "https://www.turnitin.com/" + ass["submission"]
submit_url = "https://www.turnitin.com/t_submit.asp"
query = {"aid": aid, "session-id": cookies["session-id"], "lang": "en_us"}
form_data = dict(
    async_request=1,
    author_first=author_first, 
    author_last=author_last,
    title=submission_title,
    filename="SampleSubmission.docx",
    referer=referer_url,
    userfile='',
    db_doc='',
    dropbox_filename='',
    google_doc='',
    google_auth_uri='',
    token='',
    submit_button=''
)

fd = """
------WebKitFormBoundaryTCvoHjgAffYzBXbI
Content-Disposition: form-data; name="async_request"

0
------WebKitFormBoundaryTCvoHjgAffYzBXbI
Content-Disposition: form-data; name="author_first"

Kento
------WebKitFormBoundaryTCvoHjgAffYzBXbI
Content-Disposition: form-data; name="author_last"

Nishi
------WebKitFormBoundaryTCvoHjgAffYzBXbI
Content-Disposition: form-data; name="title"

TEST DOCUMENT 1
------WebKitFormBoundaryTCvoHjgAffYzBXbI
Content-Disposition: form-data; name="userfile"; filename="SampleSubmission.docx"
Content-Type: application/vnd.openxmlformats-officedocument.wordprocessingml.document


------WebKitFormBoundaryTCvoHjgAffYzBXbI
Content-Disposition: form-data; name="db_doc"


------WebKitFormBoundaryTCvoHjgAffYzBXbI
Content-Disposition: form-data; name="dropbox_filename"


------WebKitFormBoundaryTCvoHjgAffYzBXbI
Content-Disposition: form-data; name="google_doc"


------WebKitFormBoundaryTCvoHjgAffYzBXbI
Content-Disposition: form-data; name="google_auth_uri"


------WebKitFormBoundaryTCvoHjgAffYzBXbI
Content-Disposition: form-data; name="token"


------WebKitFormBoundaryTCvoHjgAffYzBXbI
Content-Disposition: form-data; name="submit_button"


------WebKitFormBoundaryTCvoHjgAffYzBXbI--
"""


alternate_fd = [
    {"name": k, "value": v}
    for k, v in form_data.items()
]

In [24]:
[{i: form_data[i]} for i in form_data if i != "userfile"]

[{'async_request': 1},
 {'author_first': 'Kento'},
 {'author_last': 'Nishi'},
 {'title': 'Custom turnitin API'},
 {'filename': 'SampleSubmission.docx'},
 {'referer': 'https://www.turnitin.com/t_submit.asp?r=3.18340456629862&svr=41&lang=en_us&aid=93021867'},
 {'db_doc': ''},
 {'dropbox_filename': ''},
 {'google_doc': ''},
 {'google_auth_uri': ''},
 {'token': ''},
 {'submit_button': ''}]

In [47]:
r = s.post(submit_url, fd, files={"userfile": files["SampleSubmission.docx"]}, headers={"accept": "application/json"}, params=query, cookies=cookies)

ValueError: Data must not be a string.

In [17]:
r.headers["content-type"]

'application/json'

In [42]:
r.json()

{'uuid': None, 'errors': ['No file uploaded!'], 'error_codes': ['1008']}

In [12]:
rr = s.post("https://www.turnitin.com/panda/get_submission_metadata.asp", params={
    "session-id": cookies["session-id"],
    "lang": "en_us",
    "skip_ready_check": 0,
    # "uuid": NotImplemented
})

In [13]:
confirmation_url = "https://www.turnitin.com/submit_confirm.asp?"
c_query = {
    "lang": "en_us",
    "session-id": cookies["session-id"],
    "data-state": "confirm"
}
r = s.post(confirmation_url, params=query, cookies=cookies)